# Twitter User Fetch
https://developer.twitter.com/en/docs/twitter-api/early-access

### Import relevant libraries

In [3]:
import requests
import os
import json
import re

import time
import datetime

from collections import defaultdict

import matplotlib.pyplot as plt
import pandas as pd

In [147]:
#searchtweets-v2
# https://github.com/twitterdev/search-tweets-python/tree/v2

### Set static variables

In [1]:
#consumer_key = "nFLuWKu9TJJnZqFrGG1oDUjfN"
#consumer_secret = "COoAwWDANn8K8zzmqN6hJaGIPqj6oJxRmO6KCtJ7kq3ukYq6Xf"
#access_token = "1259532157703380992-ijiLvGO3gK9NlxKtZ4EkzJb2HMOxSU"
#access_token_secret = "xBM7aIrKWWsacX1GgPgZzk5181fl43bEK9fquBYuJE78G"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANxrNAEAAAAAYdSsKk0mhqqWBoM39zZCiTx5%2FBA%3DyzXLwPs1pLxTqBn8iXfKqluGsJSnKqevzbJ0l2fPvTsX4oPY6s"
#file_path = "/Users/Janiwo/Dropbox/Master thesis data/"
FILE_PATH = r"C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data"
ENDPOINT = "https://api.twitter.com/2/users/:id/following"


### Define Functions

In [40]:
def create_headers(bearer_token):
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    #print(response.headers)
    if response.status_code == 429:
        time_to_sleep = float(response.headers["x-rate-limit-reset"]) - time.time()
        print(f"Rate limit reached. Sleeping for {time_to_sleep} seconds...")
        time.sleep(time_to_sleep)
        
        response = requests.request("GET", url, headers=headers, params=params)
        print(response.status_code)
    
    elif response.status_code != 200:
        raise Exception(response.status_code, response.text)
          
    return response.json()

def write_to_json(response):
    
    with open(file, "a") as json_file:
        json_file.write(f"{json.dumps(response)}\n")

def main(endpoint):
    headers = create_headers(BEARER_TOKEN)
    json_response = connect_to_endpoint(endpoint, headers, query_params)
    write_to_json(json_response)
    #response_data_norm, response_includes_norm = create_csv(json_response)
    
    try:
        query_params["pagination_token"] = json_response["meta"]["pagination_token"]
        print(f'Next Token: {query_params["pagination_token"]}')
        time.sleep(1)
    
    except KeyError:
        global next_page
        next_page = False
        print("No further tokens available. Going to next user.")

def load_full_json(file):
    
    json_complete = {"data":list(),"users":list()}
    
    with open(file, "r") as json_file:
        for line in json_file:
            data = json.loads(line)
            json_complete["data"].extend(data["data"])
            json_complete["users"].extend(data["includes"]["users"])
    
    return json_complete

# Load User File

In [8]:
df_users = pd.read_csv(FILE_PATH + "\df_users.csv")
df_users.drop("Unnamed: 0", axis=1, inplace=True)
df_users.head()

,description,username,verified,name,created_at,id,url,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,location
0,PhD Candidate at Engineering,GaKahsay,False,Kahsay GA,2020-11-13T15:55:37.000Z,1327278886380515328,NaN,817,1215,4427,0,NaN
1,#STOPWARONTIGRAY,sayumek2020,False,Selam,2020-11-04T08:23:06.000Z,1323903491044188161,NaN,43,156,3531,0,NaN
2,ማርያም ፅዮን ምስ ወዳ ፅላል ትግራይ ትኩነልና \n#TigrayGenocid...,AbbyH06586436,False,Abby |ዓይነይ-ትግራይ እንዲኺ|,2020-11-04T23:24:06.000Z,1324130252008816640,NaN,54,232,1129,1,New Zealand
3,NaN,MahfuzHussien,False,Mahfuz Hussien,2013-01-22T22:05:12.000Z,1112761003,NaN,151,788,1455,1,NaN
4,Journalists Standing up for human rights acros...,jonewales,False,William_Sea,2020-01-02T19:56:07.000Z,1212824799107375105,NaN,315,1913,2710,3,NaN


In [10]:
df_users.shape

(1928500, 12)

In [20]:
df_users_100 = df_users[df_users["public_metrics.following_count"] >= 1000]

In [21]:
user_ids = set(df_users_100["id"])

In [22]:
len(user_ids)

452973

# General Query

In [24]:
list(user_ids)[0]

45088774

In [33]:
headers = create_headers(BEARER_TOKEN)

In [35]:
endpoint = connect_to_endpoint(f"https://api.twitter.com/2/users/{list(user_ids)[0]}/following", headers, query_params)

400


Exception: (400, '{"errors":[{"parameters":{"next_token":["H6I7E4BQB0M16ZZZ"]},"message":"The query parameter [next_token] is not one of [id,max_results,pagination_token,expansions,tweet.fields,user.fields]"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')

In [55]:
next_token = None
ctr = 0

query_params = {'max_results': 1000, #max number is 1,000
                'pagination_token': next_token}


file_ID = f'/batch1.txt'
file = FILE_PATH + file_ID


print("Starting...")
for user_id in list(user_ids)[:5]:
    username = df_users[df_users["id"] == user_id]["username"]
    following_count = df_users[df_users["id"] == user_id]["public_metrics.following_count"]
    
    next_page = True
    
    print(f"Retrieving Accounts followed by {username}. Total number of accounts: {following_count}.")
    endpoint = f"https://api.twitter.com/2/users/{user_id}/following"
    while next_page == True:
        main(endpoint)

Starting...
Retrieving Accounts followed by 1170270    alexalderthay
Name: username, dtype: object. Total number of accounts: 1170270    1044
Name: public_metrics.following_count, dtype: int64.
200
No further tokens available. End of Process.
Retrieving Accounts followed by 705681    jack
Name: username, dtype: object. Total number of accounts: 705681    4608
Name: public_metrics.following_count, dtype: int64.
200
No further tokens available. End of Process.
Retrieving Accounts followed by 1699877    NutshellsGuy
Name: username, dtype: object. Total number of accounts: 1699877    1361
Name: public_metrics.following_count, dtype: int64.
200
No further tokens available. End of Process.
Retrieving Accounts followed by 823873    tonystubblebine
Name: username, dtype: object. Total number of accounts: 823873    5780
Name: public_metrics.following_count, dtype: int64.
200
No further tokens available. End of Process.
Retrieving Accounts followed by 936071    Photojett
Name: username, dtype: o

In [43]:
file

'C:\\Users\\jawo19ad\\Dropbox (CBS)\\Master thesis data/batch1.txt'

In [49]:
json_complete = {"data":list()}
with open(file, "r") as json_file:
    for line in json_file:
        json_complete["data"].extend(data["data"])

In [54]:
json_complete["data"]

[{'id': '1192861100804575234', 'name': 'Katy', 'username': 'KatyJayne101'},
 {'id': '16044025', 'name': 'Tour of Britain 🇬🇧', 'username': 'TourofBritain'},
 {'id': '288369320',
  'name': 'Mark Cooper-Jones',
  'username': 'markcooperjones'},
 {'id': '43919376', 'name': 'Jay Foreman', 'username': 'jayforeman'},
 {'id': '19211352', 'name': 'Cherie Pridham', 'username': 'ChezPro'},
 {'id': '22621511', 'name': 'Gary Delaney', 'username': 'GaryDelaney'},
 {'id': '349083462',
  'name': 'Tirreno Adriatico',
  'username': 'TirrenAdriatico'},
 {'id': '108441763', 'name': 'Ben Swift', 'username': 'swiftybswift'},
 {'id': '3320963777', 'name': 'Conyers MFL', 'username': 'conyersmfl'},
 {'id': '3224085551',
  'name': 'Maillot à Pois E.Leclerc',
  'username': 'maillotapois'},
 {'id': '882284078', 'name': 'stycle_design', 'username': 'stycle_design'},
 {'id': '507201572', 'name': 'Fleur Faure', 'username': 'FleurFaure'},
 {'id': '3338835075',
  'name': 'Maillot Distinctif.',
  'username': 'MDistinct

In [ ]:
    json_complete = {"data":list(),"users":list()}
    
    with open(file, "r") as json_file:
        for line in json_file:
            data = json.loads(line)
            json_complete["data"].extend(data["data"])
            json_complete["users"].extend(data["includes"]["users"])

In [36]:
next_token = None
next_page = True
ctr = 0

query_params = {'max_results': 500, #max value is 500
                'pagination_token': next_token}


file_ID = f'\batch1.txt'
file = FILE_PATH + file_ID


print("Starting...")
#while next_page == True:
#    main()
#    ctr += 1
#    print(f'Retrieved Tweets: {query_params["max_results"] * ctr}')


Starting...


In [426]:
file

'/Users/Janiwo/Desktop/CBS/4. Semester/Thesis/Files/all_refugee_02_2021-02-01T00:00:00Z_2021-03-01T00:00:00Z.txt'

In [433]:
json_complete = load_full_json(file)

In [434]:
df = pd.json_normalize(json_complete["data"])

In [435]:
max(df["created_at"])

'2020-07-31T23:59:58.000Z'

In [436]:
min(df["created_at"])

'2020-07-13T19:14:52.000Z'

In [296]:
df[df["created_at"]<"2020-09-09T00:00:00.000Z"].shape

(1224, 22)

In [297]:
df[df["created_at"]>"2020-09-09T00:00:00.000Z"].shape

(26443, 22)

In [401]:
hashtag_count = get_hashtag_counts(json_complete)
sort_hash_count = sorted(hashtag_count.items(), key=lambda x: x[1], reverse=True)
sort_hash_count[:100]

[('refugees', 4335),
 ('Tigray', 3943),
 ('COVID19', 3366),
 ('StopWarOnTigray', 3203),
 ('Iraqi_Refugees_Look_For_Homeland', 3045),
 ('Ethiopia', 3002),
 ('Refugees', 2966),
 ('TigrayGenocide', 2627),
 ('migrants', 2498),
 ('immigrants', 2258),
 ('S386', 2118),
 ('UnblockS386', 2037),
 ('refugee', 1766),
 ('immigration', 1728),
 ('StopAbiy', 1652),
 ('RickBeijingScott', 1611),
 ('NoS386', 1558),
 ('Sudan', 1530),
 ('Election2020', 1450),
 ('DV2021', 1403),
 ('Immigrants', 1316),
 ('Armenia', 1257),
 ('Azerbaijan', 1170),
 ('Immigrant', 1055),
 ('BidenHarris2020', 1055),
 ('Syria', 1043),
 ('Biden', 1008),
 ('DV2020', 967),
 ('RefugeesWelcome', 966),
 ('immigrant', 935),
 ('S386OutsourcingBill', 923),
 ('Trump', 892),
 ('BJP', 879),
 ('LEGAL', 853),
 ('KarabakhisAzerbaijan', 840),
 ('EU', 829),
 ('StandWithIraqiRefugees', 800),
 ('Turkey', 792),
 ('Immigration', 777),
 ('Karabakh', 750),
 ('SmartNews', 698),
 ('DontBelieveArmenia', 695),
 ('migrant', 683),
 ('terrorist', 664),
 ('Franc